In [1]:
from dziban.mkiv import Chart
from vega_datasets import data
from copy import deepcopy
import json


In [2]:
movies = data('movies')
movies.head(1)

,Creative_Type,Director,Distributor,IMDB_Rating,IMDB_Votes,MPAA_Rating,Major_Genre,Production_Budget,Release_Date,Rotten_Tomatoes_Rating,Running_Time_min,Source,Title,US_DVD_Sales,US_Gross,Worldwide_Gross
0,None,None,Gramercy,6.1,1071.0,R,None,8000000.0,Jun 12 1998,NaN,NaN,None,The Land Girls,NaN,146083.0,146083.0


In [3]:
base = Chart(movies).field('IMDB_Rating')
base.mark('line').anchor_on(base)


['clingo', '--outf=2', '--quiet=1,2,2']
define.lp
generate.lp
hard.lp
soft.lp
weights.lp
assign_weights.lp
output.lp
compare.lp
assign_compare_weights.lp
compare_weights.lp
optimize_draco.lp
['clingo', '--outf=2', '--quiet=1,2,2', '--opt-mode=OptN', '--models=20', '--warn=no-atom-undefined']


INFO:draco.run:Constraints are unsatisfiable.


TypeError: 'NoneType' object is not iterable

In [4]:
base.mark('line').anchor_on(base)

['clingo', '--outf=2', '--quiet=1,2,2']
define.lp
generate.lp
hard.lp
soft.lp
weights.lp
assign_weights.lp
output.lp
compare.lp
assign_compare_weights.lp
compare_weights.lp
optimize_draco.lp
['clingo', '--outf=2', '--quiet=1,2,2', '--opt-mode=OptN', '--models=20', '--warn=no-atom-undefined']


INFO:draco.run:Constraints are unsatisfiable.


TypeError: 'NoneType' object is not iterable

In [5]:
base = Chart(movies)

In [6]:
q1 = 'IMDB_Rating'
q2 = 'Rotten_Tomatoes_Rating'
q3 = 'Worldwide_Gross'
n1 = 'Major_Genre'
n2 = 'MPAA_Rating'
n3 = 'Creative_Type'

fields = {
    'q': [q1, q2, q3],
    'n': [n1, n2, n3]
}

marks = ['point', 'bar', 'line', 'area', 'text', 'tick', 'rect']
aggs = ['mean']

In [7]:
def get_props_from_transform(transform):
    props = {}
    if (transform == 'bin'):
        props['bin'] = True
    elif (transform == 'agg'):
        props['aggregate'] = 'mean'
        
    return props

In [8]:
def generate_queries(currdims, targetdims, curr_query, queries):
    if (currdims < targetdims):
        for fieldtype in ['q', 'n']:
            if (fieldtype == 'q'):
                for transform in ['raw', 'bin', 'agg']:
                    next_query = deepcopy(curr_query)
                    next_query.append({ 'fieldtype': fieldtype, 'transform': transform })
                    queries.append(next_query)

                    generate_queries(currdims + 1, targetdims, next_query, queries)
            else:
                next_query = deepcopy(curr_query)
                next_query.append({ 'fieldtype': fieldtype, 'transform': 'raw' })
                queries.append(next_query)

                generate_queries(currdims + 1, targetdims, next_query, queries)

In [9]:
def dedupe_queries(queries):
    seen = set()
    
    unique = []
    for query in queries:
        reps = {}
        for field in query:
            string = '{0}({1})'.format(field['transform'], field['fieldtype'])
            if string not in reps:
                reps[string] = 0
            reps[string] += 1
            
        stringified = json.dumps(reps, sort_keys=True)
        
        if stringified not in seen:
            seen.add(stringified)
            unique.append(query)
            
    return unique
            

In [10]:
def query_to_dziban(prior, query, available_fields):
    chart = prior
    
    used_fields = {}
    
    for field in query:
        fieldtype = field['fieldtype']
        transform = field['transform']
        
        fieldname = available_fields[fieldtype].pop(0)
    
        if (fieldtype not in used_fields):
            used_fields[fieldtype] = []
        used_fields[fieldtype].append(fieldname)
        
        props = get_props_from_transform(transform)
            
        chart = chart.field(fieldname, **props)
        
    return {
        'chart': chart,
        'query': query,
        'available_fields': available_fields,
        'used_fields': used_fields
    }
        

In [11]:
priors = []
generate_queries(0, 1, [], priors)
priors = dedupe_queries(priors)

In [12]:
charts = [query_to_dziban(base, query, deepcopy(fields)) for query in priors]

In [13]:
mark_edits = []
for mark in marks:
    mark_edits.append({
        'type': 'mark',
        'mark': mark
    })

In [14]:
add_field_edits = []
for fieldtype in ['q', 'n']:
    for transform in ['raw', 'bin', 'agg']:
        add_field_edits.append({
            'type': 'add_field',
            'fieldtype': fieldtype,
            'transform': transform
        })

In [15]:
bin_edits = [
    {
        'type': 'bin'
    }
]

In [16]:
agg_edits = []
for agg in aggs:
    agg_edits.append(
        {
            'type': 'agg',
            'agg': agg
        }
    )

In [17]:
edits = mark_edits + add_field_edits + bin_edits + agg_edits

In [18]:
def edit_dziban(dzi, edit):
    edited_chart = dzi['chart']
    etype = edit['type']
    
    if etype == 'mark':
        edited_chart = edited_chart.mark(edit['mark'])
    elif etype == 'add_field':
        available_fields = dzi['available_fields']
        field = available_fields[edit['fieldtype']].pop(0)
        
        if (edit['fieldtype'] not in dzi['used_fields']):
            dzi['used_fields'][edit['fieldtype']] = []
        dzi['used_fields'][edit['fieldtype']].append(field)
        
        props = get_props_from_transform(edit['transform'])
        
        edited_chart = edited_chart.field(field, **props)
    elif etype == 'bin':
        used_fields = dzi['used_fields']
        
        if ('q' not in used_fields):
            return None
        
        field_to_bin = used_fields['q'][0]
        
        edited_chart = edited_chart.field(field_to_bin, bin=True)
    elif etype == 'agg':
        used_fields = dzi['used_fields']
        
        if ('q' not in used_fields):
            return None
        
        field_to_agg = used_fields['q'][0]
        edited_chart = edited_chart.field(field_to_agg, aggregate=edit['agg'])
        
    cold = edited_chart
    anchored = edited_chart.anchor_on(dzi['chart'])
    
    return {
        'prior': dzi['chart'],
        'edit': edit,
        'cold': cold,
        'anchored': anchored,
        'available_fields': dzi['available_fields'],
        'used_fields': dzi['used_fields']
    }

In [19]:
nexts = []
for chart in charts:
    for edit in edits:
        prior = deepcopy(chart)
        edited = edit_dziban(prior, edit)
        nexts.append(edited)

In [20]:
with_differences = []
for (i, n) in enumerate(nexts):
    print('{0} / {1}'.format(i, len(nexts)))
    
    cold = n['cold']
    anchored = n['anchored']
    
    if not cold.is_satisfiable():
        print('cold unsat')
        continue
        
    if not anchored.is_satisfiable():
        print('anchored unsat')
        print(n['prior']._get_vegalite())
        print(n['edit'])
        print('\n'.join(n['anchored']._get_full_query()))
        print(n['cold']._get_asp_complete())
        break
        
    if i == 2:
        break
    
    diff = n['cold'] - n['anchored']
    if (len(diff) != 0):
        with_differences.append(n)

0 / 60
['clingo', '--outf=2', '--quiet=1,2,2']
['clingo', '--outf=2', '--quiet=1,2,2']
define.lp
generate.lp
hard.lp
soft.lp
weights.lp
assign_weights.lp
output.lp
compare.lp
assign_compare_weights.lp
compare_weights.lp
optimize_draco.lp
['clingo', '--outf=2', '--quiet=1,2,2', '--opt-mode=OptN', '--models=20', '--warn=no-atom-undefined']


ERROR:draco.run:stdout: b'{\n  "Solver": "clingo version 5.3.0",\n  "Input": [\n    "stdin"\n  ],\n  "Call": [\n    {\n      "Witnesses": [\n        {\n          "Value": [\n            "base(\\"anchor0\\")", "visualization(\\"view\\")", "visualization(\\"anchor0\\")", "encoding(\\"view\\",e0)", "encoding(\\"anchor0\\",e0)", "mark(\\"view\\",point)", "mark(\\"anchor0\\",tick)", "zero(\\"anchor0\\",e0)", "channel(\\"anchor0\\",e0,x)", "field(\\"anchor0\\",e0,\\"IMDB_Rating\\")", "soft(encoding,\\"view\\",e0)", "soft(encoding,\\"anchor0\\",e0)", "soft(encoding_field,\\"anchor0\\",e0)", "soft(type_q,\\"anchor0\\",e0)", "soft(continuous_x,\\"anchor0\\",e0)", "type(\\"anchor0\\",e0,quantitative)", "compare(edit_point_tick,\\"anchor0\\",\\"view\\",tick,point)", "soft(c_d_tick,\\"anchor0\\",_placeholder)", "soft_weight(c_d_tick,0)", "compare_weight(edit_point_tick,1)", "soft_weight(encoding,0)", "soft_weight(encoding_field,6)", "soft_weight(type_q,0)", "soft_weight(continuous_x,0)", "channel(

ERROR:draco.run:stderr: b''


anchored unsat
['clingo', '--outf=2', '--quiet=1,2,2']
{'$schema': 'https://vega.github.io/schema/vega-lite/v3.json', 'data': {'url': 'data/cars.json'}, 'mark': 'tick', 'encoding': {'x': {'type': 'quantitative', 'field': 'IMDB_Rating', 'scale': {'zero': True}}}}
{'type': 'mark', 'mark': 'point'}
['clingo', '--outf=2', '--quiet=1,2,2']
visualization("view").
num_rows(3201).

fieldtype("Creative_Type",string).
cardinality("Creative_Type", 10).
fieldtype("Director",string).
cardinality("Director", 551).
fieldtype("Distributor",string).
cardinality("Distributor", 175).
fieldtype("IMDB_Rating",number).
cardinality("IMDB_Rating", 78).
fieldtype("IMDB_Votes",number).
cardinality("IMDB_Votes", 2840).
fieldtype("MPAA_Rating",string).
cardinality("MPAA_Rating", 8).
fieldtype("Major_Genre",string).
cardinality("Major_Genre", 13).
fieldtype("Production_Budget",number).
cardinality("Production_Budget", 382).
fieldtype("Release_Date",string).
cardinality("Release_Date", 1600).
fieldtype("Rotten_Toma

In [21]:
nexts[2]['anchored']

['clingo', '--outf=2', '--quiet=1,2,2']
define.lp
generate.lp
hard.lp
soft.lp
weights.lp
assign_weights.lp
output.lp
compare.lp
assign_compare_weights.lp
compare_weights.lp
optimize_draco.lp
['clingo', '--outf=2', '--quiet=1,2,2', '--opt-mode=OptN', '--models=20', '--warn=no-atom-undefined']


INFO:draco.run:Constraints are unsatisfiable.


TypeError: 'NoneType' object is not iterable